# NannyML Workflow

In [ ]:
!rm *.svg
!ls -la

In [ ]:
import nannyml as nml
import pandas as pd

In [ ]:
nml.__version__

In [ ]:
# Function that helps add results to RST docs.
def show_df_rst(df):
    """
    Show a dataframe as rst output:
    """
    print(df.head().to_markdown(tablefmt="grid"))

In [ ]:
reference, analysis, analysis_gt = nml.load_synthetic_sample()
show_df_rst(reference)

In [ ]:
md = nml.extract_metadata(data = reference, model_name='wfh_predictor')

In [ ]:
md.print()

In [ ]:
md.timestamp_column_name = 'timestamp'
# md.prediction_column_name = 'y_pred_proba'
md.target_column_name = 'work_home_actual'

In [ ]:
md.print()

In [ ]:
univariate_calculator = nml.UnivariateStatisticalDriftCalculator(model_metadata=md, chunk_size=5000)

In [ ]:
univariate_calculator.fit(reference_data=reference)

In [ ]:
fdata = pd.concat([reference, analysis], ignore_index=True)
fdata

In [ ]:
univariate_results = univariate_calculator.calculate(data=fdata)

In [ ]:
univariate_results_columns = list(univariate_results.columns)
univariate_results_columns

In [ ]:
show_df_rst(univariate_results.iloc[5:, :9])

In [ ]:
ranking = nml.Ranker.by('alert_count').rank(univariate_results, only_drifting=True)
ranking

In [ ]:
rcerror_calculator = nml.DataReconstructionDriftCalculator(model_metadata=md, chunk_size=5000)

In [ ]:
rcerror_calculator.fit(reference_data=reference)

In [ ]:
rcerror_results = rcerror_calculator.calculate(data=fdata)
rcerror_results

In [ ]:
plots = nml.DriftPlots(model_metadata=univariate_calculator.model_metadata, chunker=univariate_calculator.chunker)

In [ ]:
for itm in md.features:

    fig = plots.plot_univariate_statistical_drift(univariate_results, metric='statistic', feature_label=itm.label)
    fig.show()
    fig.write_image(file=f"drift-guide-{itm.label}.svg")

In [ ]:
for itm in md.continuous_features:

    fig = plots.plot_continuous_feature_distribution_over_time(
        data=pd.concat([reference, analysis], ignore_index=True),
        drift_results=univariate_results,
        feature_label=itm.label
    )
    fig.show()
    fig.write_image(file=f"drift-guide-joyplot-{itm.label}.svg")

In [ ]:
for itm in md.categorical_features:

    fig = plots.plot_categorical_feature_distribution_over_time(
        data=pd.concat([reference, analysis], ignore_index=True),
        drift_results=univariate_results,
        feature_label=itm.label
    )
    fig.show()
    fig.write_image(file=f"drift-guide-stacked-{itm.label}.svg")

In [ ]:
fig = plots.plot_data_reconstruction_drift(rcerror_results)
fig.show()
fig.write_image(file=f"drift-guide-multivariate.svg")

In [ ]:
fig = plots.plot_univariate_statistical_prediction_drift(univariate_results, metric='statistic')
fig.show()
fig.write_image('drift-guide-predictions.svg')

In [ ]:
fig = plots.plot_prediction_distribution_over_time(
    data=pd.concat([reference, analysis], ignore_index=True),
    drift_results=univariate_results)
fig.show()
fig.write_image('drift-guide-predictions-joyplot.svg')

In [ ]:
# fit estimator and estimate
cbpe = nml.CBPE(model_metadata=md, chunk_size=5000)
cbpe.fit(reference_data=reference)
est_perf = cbpe.estimate(data=fdata)

In [ ]:
# show results
plots = nml.PerformancePlots(model_metadata=md, chunker=cbpe.chunker)
fig = plots.plot_cbpe_performance_estimation(est_perf)
fig.show()
fig.write_image(file=f"perf-est-guide-syth-example.svg")